TODO: document this

things to note: 
- model was trained on GPU so need to load weights and compile to CPU
- important to check torchvision (if used here) and torch versions & be sure they match the versions pinned in the deployment container's Dockerfile

In [ ]:
import torch
from efficientnet_pytorch import EfficientNet

torch.__version__

In [ ]:
ckpt = torch.load("model-weights/ckpt_18.pt", map_location=torch.device("cpu"))
model = EfficientNet.from_pretrained("efficientnet-b3", num_classes=5)
model.load_state_dict(ckpt['model'])

In [ ]:
model.eval()

In [ ]:
# from https://github.com/microsoft/CameraTraps/blob/ccb5e98095cf81a625bf19129cb3dc97354f6284/classification/evaluate_model.py#L71
img_size = 300
compiled_path = './model-weights/mira_compiled_cpu.pt'

model.set_swish(memory_efficient=False)
ex_img = torch.rand(1, 3, img_size, img_size)
scripted_model = torch.jit.trace(model, (ex_img,))

scripted_model.save(compiled_path)
print('Saved TorchScript compiled model to', compiled_path)